# CNN_BiRNN for Speech Enhancement (Spectrogram)

In [3]:
import sys
## To get helper functions from another folder
# sys.path.insert(0, '../') # if _helper_basics_ is in previous folder
now_i_am_at = 'home' # home dso test
if now_i_am_at=='home': sys.path.insert(0, 'E:/Leonard HDD/Dropbox/DSO/Tasks/')
elif now_i_am_at=='dso': sys.path.insert(0, 'D:/Dropbox/DSO/Tasks')

from _helper_basics_ import *
from _helper_enhancement_ import *

%matplotlib inline
plt.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

print('keras',      keras.__version__)
print('tensorflow', tf.__version__)

keras 2.0.8
tensorflow 1.3.0


# Data

In [4]:
sr = 16000

## Original wavfiles
data_dir = 'E:/Leonard HDD/Dropbox/Speech Audio Text/3) Dataset/Reverb_2'
noisy_dir = os.path.join(data_dir, 'noisy_trainset_28spk_wav')
clean_dir = os.path.join(data_dir, 'clean_trainset_28spk_wav')
print(noisy_dir)
print(clean_dir)

## Saving
pwd = 'E:/Leonard HDD/Dropbox/DSO/Tasks/Dereverberation'
save_data_dir = os.path.join(pwd, 'Reverb_Data_28spk_wav')
Noise_Mag_dir = os.path.join(save_data_dir, 'Noise_Mag')
Noise_Pha_dir = os.path.join(save_data_dir, 'Noise_Pha')
Clean_Mag_dir = os.path.join(save_data_dir, 'Clean_Mag')
Clean_Pha_dir = os.path.join(save_data_dir, 'Clean_Pha')
print('save_data_dir',save_data_dir)
print('Noise_Mag_dir',Noise_Mag_dir)
print('Noise_Pha_dir',Noise_Pha_dir)
print('Clean_Mag_dir',Clean_Mag_dir)
print('Clean_Pha_dir',Clean_Pha_dir)

noise_files_list = []
for noise_wav in glob.glob( os.path.join(noisy_dir, '*.wav') ):
    noise_wav_term = noise_wav[len(noisy_dir)+1:len(noise_wav)-4]
    noise_files_list.append(noise_wav_term)
len(noise_files_list)

E:/Leonard HDD/Dropbox/Speech Audio Text/3) Dataset/Reverb_2\noisy_trainset_28spk_wav
E:/Leonard HDD/Dropbox/Speech Audio Text/3) Dataset/Reverb_2\clean_trainset_28spk_wav
save_data_dir E:/Leonard HDD/Dropbox/DSO/Tasks/Dereverberation\Reverb_Data_28spk_wav
Noise_Mag_dir E:/Leonard HDD/Dropbox/DSO/Tasks/Dereverberation\Reverb_Data_28spk_wav\Noise_Mag
Noise_Pha_dir E:/Leonard HDD/Dropbox/DSO/Tasks/Dereverberation\Reverb_Data_28spk_wav\Noise_Pha
Clean_Mag_dir E:/Leonard HDD/Dropbox/DSO/Tasks/Dereverberation\Reverb_Data_28spk_wav\Clean_Mag
Clean_Pha_dir E:/Leonard HDD/Dropbox/DSO/Tasks/Dereverberation\Reverb_Data_28spk_wav\Clean_Pha


11572

# Setup & Training

In [5]:
Archi_dir = "v6__CNNBiRNN_deeper/"
Weights_path = Archi_dir+"Logs/"
Ckpt_Mod_Weights_fold = Archi_dir+"Checkpoint_Model_Weights/"
plot_path_dir = Archi_dir+'Plots/'
if not os.path.exists(Archi_dir): os.mkdir(Archi_dir)
if not os.path.exists(Weights_path): os.mkdir(Weights_path)
if not os.path.exists(Ckpt_Mod_Weights_fold): os.mkdir(Ckpt_Mod_Weights_fold)
if not os.path.exists(plot_path_dir): os.mkdir(plot_path_dir)

### debug model

In [6]:
num_freq,num_time,num_channel    = 129,128,1

filt_incept_list = [[16, 8, 4, 16, 8], 
                    [16, 8, 4, 16, 8],
                    [16, 8, 4, 16, 8], 
                    [16, 8, 4, 16, 8],
                    [16, 8, 4, 16, 8], 
                    [16, 8, 4, 16, 8],
                    [16, 8, 4, 16, 8], 
                    [16, 8, 4, 16, 8],
                    [16, 8, 4, 16, 8], 
                    [16, 8, 4, 16, 8],
                    [16, 8, 4, 16, 8], 
                    [16, 8, 4, 16, 8],
                    [16, 8, 4, 16, 8], 
                    [16, 8, 4, 16, 8]]

# filt_incept_list = [[16, 8, 4, 16, 8], 
#                     [32, 16, 8, 32, 16],
#                     [32, 16, 8, 32, 16], 
#                     [32, 16, 8, 32, 16],
#                     [32, 16, 8, 32, 16], 
#                     [32, 16, 8, 32, 16],
#                     [32, 16, 8, 32, 16], 
#                     [16, 8, 4, 16, 8]]

# filt_incept_list = [[16, 8, 8, 16, 16], 
#                     [32, 16, 16, 32, 32],
#                     [32, 16, 16, 32, 32], 
#                     [16, 8, 8, 16, 16]]
rnn_neu_list = [num_freq,num_freq]
########################### Printing information ###########################
build_model(num_freq,num_time,num_channel, filt_incept_list, rnn_neu_list,reg_l2=0.).summary()
# model = build_model(num_freq,num_time,num_channel, filt_incept_list, rnn_neu_list,reg_l2=0.)
# model.summary()
# print_model_weights(model)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Input (InputLayer)               (None, 129, 128, 1)   0                                            
____________________________________________________________________________________________________
0_1b_Conv2D (Conv2D)             (None, 129, 128, 8)   16          Input[0][0]                      
____________________________________________________________________________________________________
0_1c_Conv2D (Conv2D)             (None, 129, 128, 4)   8           Input[0][0]                      
____________________________________________________________________________________________________
0_1a_Conv2D (Conv2D)             (None, 129, 128, 16)  32          Input[0][0]                      
___________________________________________________________________________________________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 129)          0         
_________________________________________________________________
0_Bi_GRU (Bidirectional)     (None, 128, 258)          200466    
_________________________________________________________________
1_GRU (GRU)                  (None, 128, 129)          150156    
Total params: 350,622
Trainable params: 350,622
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 129, 128, 1)       0         
_________________________________________________________________
Conv_SubNet (Model)          (None, 129, 128, 1)       43005     
_________________________________________________________________
resh

# Training Params

In [16]:
Folder_det = [noisy_dir,clean_dir,Noise_Mag_dir,Noise_Pha_dir,Clean_Mag_dir,Clean_Pha_dir]
num_Det = [num_freq,num_time,num_channel]
######################## Training Parameters ###############################
decay = 0;          	print('decay',decay)
steps_per_epoch = 500;	print('steps_per_epoch',steps_per_epoch)
epochs = 150;			print('epochs',epochs)
batch_size = 8;   		print('batch_size',batch_size)

decay 0
steps_per_epoch 500
epochs 150
batch_size 8


# V6
## 14 Inception + 1 Conv + 1 BiGRU + 1 GRU


#### Initial training : train_model_v6_1

In [ ]:
filt_incept_list = [[16, 8, 4, 16, 8]]*14
rnn_neu_list = [num_freq,num_freq]
v6_1 = build_model(num_freq=129,num_time=128,num_channel=1, filt_incept_list=filt_incept_list, rnn_neu_list=rnn_neu_list, verbose=False)
v6_1.summary()
######################## Training Parameters ###############################
learning_rate = 1e-3;   print('learning_rate',learning_rate)
adam_opt = Adam(lr=learning_rate, decay=decay)
v6_1.compile(loss='mse' ,optimizer=adam_opt)
######################## Checkpoints ###############################
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, min_lr=1e-6, cooldown=5, epsilon=1e-05, verbose=1)
CkptFold_det = [  'v6_1',    Ckpt_Mod_Weights_fold, 'v6_1/']
ModelCheckpoint_det = ['loss',     1,            True,              True,           1] 
ckpt = ckpt_saving(CkptFold_det, ModelCheckpoint_det, save_all=True)
csv_log = CSVLogger(Weights_path+'v6_1_Trglog.txt', '\t', append=False)
plot_path = plot_path_dir+'v6_1.png'
########################### Actual training ##################################
try:
    history_v6_1 = v6_1.fit_generator( infinite_Gen(batch_size,noise_files_list,Folder_det,num_Det),
#                            steps_per_epoch=2, epochs=2, verbose=1, 
                           steps_per_epoch=steps_per_epoch, epochs=100, verbose=1, 
                           callbacks=[reduce_lr, csv_log, ckpt], 
                           validation_data=None, validation_steps=None, shuffle=True, 
                           initial_epoch=0)
    plot_loss(history_v6_1, metric_list=['loss'], title='Loss',  plot_path=plot_path)
except KeyboardInterrupt:
    print('\n\nKeyboardInterrupt')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 129, 128, 1)       0         
_________________________________________________________________
Conv_SubNet (Model)          (None, 129, 128, 1)       43005     
_________________________________________________________________
reshape_13 (Reshape)         (None, 129, 128)          0         
_________________________________________________________________
permute_13 (Permute)         (None, 128, 129)          0         
_________________________________________________________________
Bi_RNN_SubNet (Model)        (None, 128, 129)          350622    
_________________________________________________________________
permute_14 (Permute)         (None, 129, 128)          0         
_________________________________________________________________
reshape_14 (Reshape)         (None, 129, 128, 1)       0         
Total para

194/500 [==========>...................] - ETA: 525s - loss: 0.3277p239_290


301/500 [=================>............] - ETA: 339s - loss: 0.2993p278_047


408/500 [=======================>......] - ETA: 156s - loss: 0.2849p254_239


499/500 [============================>.] - ETA: 1s - loss: 0.2720p268_215
Epoch 00000: loss improved from inf to 0.27168, saving model to v6__CNNBiRNN_deeper/Checkpoint_Model_Weights/v6_1/weights_v6_1_Epoch-0000_L-0.27.hdf5
500/500 [==============================] - 849s - loss: 0.2717   
Epoch 2/100
p259_306
 14/500 [..............................] - ETA: 824s - loss: 0.2796p239_305


121/500 [======>.......................] - ETA: 637s - loss: 0.1770p243_003


228/500 [============>.................] - ETA: 459s - loss: 0.1641p228_084


274/500 [===============>..............] - ETA: 381s - loss: 0.1553p228_272


In [ ]:
save_or_load = 'save'
model_path = Weights_path+Archi_dir[:-1]+"_model_v6_1.json"
weights_path = Weights_path+Archi_dir[:-1]+"_weights_v6_1.h5"
if save_or_load == 'save':
    save_model(v6_1, model_path, weights_path)
elif save_or_load == 'load':
    v6_1 = load_model(model_path, 'path', weights_path, 'final')
    v6_1.summary()

#### Further training : train_model_v6_2

In [ ]:
save_or_load = 'load'
model_path = Weights_path+Archi_dir[:-1]+"_model_v6_1.json"
weights_path = Weights_path+Archi_dir[:-1]+"_weights_v6_1.h5"
if save_or_load == 'save':
    save_model(v6_1, model_path, weights_path)
elif save_or_load == 'load':
    v6_2 = load_model(model_path, 'path', weights_path, 'final')
    v6_1,history_v6_1 = 0,0
######################## Training Parameters ###############################
learning_rate = ;   print('learning_rate',learning_rate)
adam_opt = Adam(lr=learning_rate, decay=decay)
v6_2.compile(loss='mse' ,optimizer=adam_opt)
######################## Checkpoints ###############################
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, min_lr=1e-6, cooldown=5, epsilon=1e-05, verbose=1)
CkptFold_det = [  'v6_2',    Ckpt_Mod_Weights_fold, 'v6_2/']
ModelCheckpoint_det = ['loss',     1,            True,              True,           1] 
ckpt = ckpt_saving(CkptFold_det, ModelCheckpoint_det, save_all=True)
csv_log = CSVLogger(Weights_path+'v6_2_Trglog.txt', '\t', append=False)
plot_path = plot_path_dir+'v6_2.png'
########################### Actual training ##################################
try:
    history_v6_2 = v6_2.fit_generator( infinite_Gen(batch_size,noise_files_list,Folder_det,num_Det),
                           steps_per_epoch=steps_per_epoch, epochs=100, verbose=1, 
                           callbacks=[reduce_lr, csv_log, ckpt], 
                           validation_data=None, validation_steps=None, shuffle=True, 
                           initial_epoch=100)
    plot_loss(history_v6_2, metric_list=['loss'], title='Loss',  plot_path=plot_path)
except KeyboardInterrupt:
    print('\n\nKeyboardInterrupt')

In [132]:
save_or_load = 'save'
model_path = Weights_path+Archi_dir[:-1]+"_model_v1_2.json"
weights_path = Weights_path+Archi_dir[:-1]+"_weights_v1_2.h5"
if save_or_load == 'save':
    save_model(train_model_v1_2, model_path, weights_path)
elif save_or_load == 'load':
    train_model_v1_2 = load_model(model_path, 'path', weights_path, 'final')
    train_model_v1_2.summary()

Saved model : v1/Logs/v1_model_v1_2.json
Saved weights : v1/Logs/v1_weights_v1_2.h5
